In [13]:
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

from chart_studio import plotly
import plotly.offline as pyoff
import plotly.graph_objs as go
from plotly.subplots import make_subplots

from pmdarima import auto_arima
from sklearn.base import BaseEstimator
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

In [10]:
df = pd.read_excel(r'C:\Users\MinhDH\Downloads\kamereo_test.xlsx')
df.set_index('Date', inplace=True)
df

,Category A,Customer,Qty Sold,Revenue
Date,,,,
20171001,A,2,18,4058699
20171001,A,1,29,13178000
20171002,A,1,55,23493200
20171002,A,2,30,6540835
20171003,A,2,25,5888624
...,...,...,...,...
20180928,A,1,106,88288800
20180929,A,2,183,33435608
20180929,A,1,79,70059570


In [4]:


class AutoArimaEstimator(BaseEstimator):
    def __init__(self, **kwargs):
        self.kwargs = kwargs

    def fit(self, X, y=None):
        return auto_arima(X, seasonal=True, m=12, trace=True, error_action='ignore', suppress_warnings=True, **self.kwargs)

    def predict(self, X, n_periods):
        model = self.fit(X)
        return model.predict(n_periods=n_periods)

    def score(self, X, y, **kwargs):
        y_pred = self.predict(X, len(y))
        return -mean_squared_error(y, y_pred)


# Tạo estimator
estimator_customers = AutoArimaEstimator(order=(2, 1, 2), seasonal_order=(1, 0, 1, 12))
estimator_sales = AutoArimaEstimator(order=(2, 1, 2), seasonal_order=(1, 0, 1, 12))
estimator_revenue = AutoArimaEstimator(order=(2, 1, 2), seasonal_order=(1, 0, 1, 12))

# Tìm kiếm tham số cho số lượng khách hàng
grid_customers = GridSearchCV(estimator_customers, param_grid={}, scoring='neg_mean_squared_error')
grid_customers.fit(df['Customer'], df['Customer'])

grid_sales = GridSearchCV(estimator_sales, param_grid={}, scoring='neg_mean_squared_error')
grid_sales.fit(df['Qty Sold'], df['Qty Sold'])

grid_revenue = GridSearchCV(estimator_revenue, param_grid={}, scoring='neg_mean_squared_error')
grid_revenue.fit(df['Revenue'], df['Revenue'])

# Dự báo
forecast_customers = grid_customers.best_estimator_.predict(df['Customer'], n_periods=60)
forecast_sales = grid_sales.best_estimator_.predict(df['Qty Sold'], n_periods=60)
forecast_revenue = grid_revenue.best_estimator_.predict(df['Revenue'], n_periods=60)



Performing stepwise search to minimize aic
 ARIMA(2,0,2)(1,0,1)[12] intercept   : AIC=inf, Time=2.29 sec
 ARIMA(0,0,0)(0,0,0)[12] intercept   : AIC=851.724, Time=0.05 sec
 ARIMA(1,0,0)(1,0,0)[12] intercept   : AIC=681.315, Time=0.57 sec
 ARIMA(0,0,1)(0,0,1)[12] intercept   : AIC=inf, Time=1.32 sec
 ARIMA(0,0,0)(0,0,0)[12]             : AIC=2194.434, Time=0.01 sec
 ARIMA(1,0,0)(0,0,0)[12] intercept   : AIC=681.292, Time=0.03 sec
 ARIMA(1,0,0)(0,0,1)[12] intercept   : AIC=681.255, Time=0.21 sec
 ARIMA(1,0,0)(1,0,1)[12] intercept   : AIC=684.720, Time=1.35 sec
 ARIMA(1,0,0)(0,0,2)[12] intercept   : AIC=683.125, Time=0.58 sec
 ARIMA(1,0,0)(1,0,2)[12] intercept   : AIC=inf, Time=2.68 sec
 ARIMA(0,0,0)(0,0,1)[12] intercept   : AIC=852.278, Time=0.11 sec
 ARIMA(2,0,0)(0,0,1)[12] intercept   : AIC=616.970, Time=0.21 sec
 ARIMA(2,0,0)(0,0,0)[12] intercept   : AIC=616.719, Time=0.08 sec
 ARIMA(2,0,0)(1,0,0)[12] intercept   : AIC=617.042, Time=1.10 sec
 ARIMA(2,0,0)(1,0,1)[12] intercept   : AIC=6

c:\Users\MinhDH\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\MinhDH\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 982, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\MinhDH\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_scorer.py", line 253, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\MinhDH\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_scorer.py", line 345, in _score
    y_pred = method_calle

 ARIMA(2,0,2)(1,0,1)[12] intercept   : AIC=inf, Time=2.16 sec
 ARIMA(0,0,0)(0,0,0)[12] intercept   : AIC=851.724, Time=0.05 sec
 ARIMA(1,0,0)(1,0,0)[12] intercept   : AIC=665.488, Time=0.82 sec
 ARIMA(0,0,1)(0,0,1)[12] intercept   : AIC=inf, Time=1.35 sec
 ARIMA(0,0,0)(0,0,0)[12]             : AIC=2194.434, Time=0.01 sec
 ARIMA(1,0,0)(0,0,0)[12] intercept   : AIC=667.289, Time=0.03 sec
 ARIMA(1,0,0)(2,0,0)[12] intercept   : AIC=667.211, Time=2.96 sec
 ARIMA(1,0,0)(1,0,1)[12] intercept   : AIC=668.982, Time=1.09 sec
 ARIMA(1,0,0)(0,0,1)[12] intercept   : AIC=665.640, Time=0.23 sec
 ARIMA(1,0,0)(2,0,1)[12] intercept   : AIC=669.228, Time=3.50 sec
 ARIMA(0,0,0)(1,0,0)[12] intercept   : AIC=853.473, Time=0.27 sec
 ARIMA(2,0,0)(1,0,0)[12] intercept   : AIC=609.097, Time=1.05 sec
 ARIMA(2,0,0)(0,0,0)[12] intercept   : AIC=608.968, Time=0.12 sec
 ARIMA(2,0,0)(0,0,1)[12] intercept   : AIC=609.191, Time=0.49 sec
 ARIMA(2,0,0)(1,0,1)[12] intercept   : AIC=611.252, Time=1.25 sec
 ARIMA(3,0,0)(0,0

c:\Users\MinhDH\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\MinhDH\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 982, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\MinhDH\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_scorer.py", line 253, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\MinhDH\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_scorer.py", line 345, in _score
    y_pred = method_calle

 ARIMA(2,0,2)(1,0,1)[12] intercept   : AIC=497.617, Time=2.63 sec
 ARIMA(0,0,0)(0,0,0)[12] intercept   : AIC=851.724, Time=0.07 sec
 ARIMA(1,0,0)(1,0,0)[12] intercept   : AIC=683.102, Time=0.65 sec
 ARIMA(0,0,1)(0,0,1)[12] intercept   : AIC=inf, Time=1.29 sec
 ARIMA(0,0,0)(0,0,0)[12]             : AIC=2194.434, Time=0.01 sec
 ARIMA(2,0,2)(0,0,1)[12] intercept   : AIC=472.681, Time=2.08 sec
 ARIMA(2,0,2)(0,0,0)[12] intercept   : AIC=474.463, Time=0.78 sec
 ARIMA(2,0,2)(0,0,2)[12] intercept   : AIC=469.797, Time=4.46 sec
 ARIMA(2,0,2)(1,0,2)[12] intercept   : AIC=655.437, Time=5.28 sec
 ARIMA(1,0,2)(0,0,2)[12] intercept   : AIC=inf, Time=3.80 sec
 ARIMA(2,0,1)(0,0,2)[12] intercept   : AIC=497.685, Time=2.88 sec
 ARIMA(3,0,2)(0,0,2)[12] intercept   : AIC=490.595, Time=5.67 sec
 ARIMA(2,0,3)(0,0,2)[12] intercept   : AIC=466.739, Time=4.13 sec
 ARIMA(2,0,3)(0,0,1)[12] intercept   : AIC=466.941, Time=2.03 sec
 ARIMA(2,0,3)(1,0,2)[12] intercept   : AIC=489.683, Time=9.54 sec
 ARIMA(2,0,3)(1,0

...

c:\Users\MinhDH\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\MinhDH\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


Performing stepwise search to minimize aic
 ARIMA(2,1,2)(1,0,1)[12] intercept   : AIC=7098.905, Time=1.32 sec
 ARIMA(0,1,0)(0,0,0)[12] intercept   : AIC=7445.296, Time=0.02 sec
 ARIMA(1,1,0)(1,0,0)[12] intercept   : AIC=7220.742, Time=0.29 sec
 ARIMA(0,1,1)(0,0,1)[12] intercept   : AIC=7139.545, Time=0.28 sec
 ARIMA(0,1,0)(0,0,0)[12]             : AIC=7443.315, Time=0.01 sec
 ARIMA(2,1,2)(0,0,1)[12] intercept   : AIC=7097.008, Time=1.27 sec
 ARIMA(2,1,2)(0,0,0)[12] intercept   : AIC=7097.985, Time=0.56 sec
 ARIMA(2,1,2)(0,0,2)[12] intercept   : AIC=7098.892, Time=3.02 sec
 ARIMA(2,1,2)(1,0,0)[12] intercept   : AIC=7097.100, Time=1.25 sec
 ARIMA(2,1,2)(1,0,2)[12] intercept   : AIC=inf, Time=4.33 sec
 ARIMA(1,1,2)(0,0,1)[12] intercept   : AIC=7143.438, Time=0.75 sec
 ARIMA(2,1,1)(0,0,1)[12] intercept   : AIC=7097.146, Time=1.01 sec
 ARIMA(3,1,2)(0,0,1)[12] intercept   : AIC=7096.797, Time=1.12 sec
 ARIMA(3,1,2)(0,0,0)[12] intercept   : AIC=7097.686, Time=0.56 sec
 ARIMA(3,1,2)(1,0,1)[12]

c:\Users\MinhDH\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\MinhDH\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


 ARIMA(2,1,2)(1,0,1)[12] intercept   : AIC=inf, Time=1.53 sec
 ARIMA(0,1,0)(0,0,0)[12] intercept   : AIC=27887.352, Time=0.01 sec
 ARIMA(1,1,0)(1,0,0)[12] intercept   : AIC=27602.979, Time=0.14 sec
 ARIMA(0,1,1)(0,0,1)[12] intercept   : AIC=27353.464, Time=0.23 sec
 ARIMA(0,1,0)(0,0,0)[12]             : AIC=27885.353, Time=0.01 sec
 ARIMA(0,1,1)(0,0,0)[12] intercept   : AIC=27351.634, Time=0.05 sec
 ARIMA(0,1,1)(1,0,0)[12] intercept   : AIC=27353.256, Time=0.20 sec
 ARIMA(0,1,1)(1,0,1)[12] intercept   : AIC=inf, Time=0.66 sec
 ARIMA(1,1,1)(0,0,0)[12] intercept   : AIC=27344.541, Time=0.11 sec
 ARIMA(1,1,1)(1,0,0)[12] intercept   : AIC=27346.021, Time=0.38 sec
 ARIMA(1,1,1)(0,0,1)[12] intercept   : AIC=27346.182, Time=0.41 sec
 ARIMA(1,1,1)(1,0,1)[12] intercept   : AIC=inf, Time=1.02 sec
 ARIMA(1,1,0)(0,0,0)[12] intercept   : AIC=27602.323, Time=0.03 sec
 ARIMA(2,1,1)(0,0,0)[12] intercept   : AIC=27345.287, Time=0.14 sec
 ARIMA(1,1,2)(0,0,0)[12] intercept   : AIC=27340.391, Time=0.22 se

c:\Users\MinhDH\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\MinhDH\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [7]:
# Tạo Dataframe
forecast_dates = pd.date_range(start='2024-10-01', periods=60)
forecast_df = pd.DataFrame({
    'Date': forecast_dates,
    'Customers': forecast_customers,
    'Sales': forecast_sales,
    'Revenue': forecast_revenue
})

forecast_df
# forecast_df.to_excel("Kamereo_Forcast.xlsx", index = False)

,Date,Customers,Sales,Revenue
730,2024-10-01,1.447373,134.036088,6.193326e+07
731,2024-10-02,1.537294,140.257068,5.408909e+07
732,2024-10-03,1.451584,147.494527,5.617277e+07
733,2024-10-04,1.555762,143.048796,5.970770e+07
734,2024-10-05,1.511371,144.617453,5.372747e+07
735,2024-10-06,1.486676,151.447286,5.813100e+07
736,2024-10-07,1.523953,146.347194,5.860665e+07
737,2024-10-08,1.467895,152.277668,5.413023e+07
738,2024-10-09,1.540469,147.824359,5.956791e+07
739,2024-10-10,1.452490,154.763827,5.732371e+07


Sử dụng mô hình SARIMA (Autoregressive Integrated Moving Average) để thống kê các dữ liệu quá khứ để dự báo chuối thời gian. Sử dụng các yếu tố chính là chuỗi tự hồi quy AR (auto regression), trung bình động (Moving Average) và bậc của sai phân để thể hiện tính dừng, Như vậy các tham số sẽ bao gôm ARIMA(p, d, q). Việc lựa chọn các tham số phù hợp cho mô hình chủ yếu dựa trên chỉ số AIC. Do đó mô hình này sử dụng phương pháp auto arima để có thể tự động thực hiện quy trình. Sử dụng một grid search và chúng ta truyền vào các hệ số giới hạn của các bậc (p,d,q), sau đó thuật toán sẽ đưa ra một mô hình tối ưu nhất.

In [16]:
forecast_df['Date'] = pd.to_datetime(forecast_df['Date']).dt.date
fig = make_subplots(rows=1, cols=1, specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Bar(
        x=forecast_df['Date'],
        y=forecast_df['Revenue'],
        name='Revenue'),
    secondary_y=False
)


fig.update_layout(
    xaxis={"title": "Date", "type": "category"},
    yaxis={"title": "Revenue"},
    title='Revenue Forecast'
)


fig.show()


In [22]:
fig = make_subplots(rows=1, cols=1, specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Bar(
        x=forecast_df['Date'],
        y=forecast_df['Sales'],
        name='Quantity Sold'),
    secondary_y=False
)
fig.add_trace(
    go.Scatter(
        x=forecast_df['Date'],
        y=forecast_df['Customers'],
        mode='lines+markers',
        name='Number of Customer',
        marker=dict(
            color='rgb(255, 140, 0)',
            size=8
    )
    ),
    secondary_y=True
)

fig.update_layout(
    xaxis={"title": "Date", "type": "category"},
    yaxis={"title": "Quantity Sold"},
    yaxis2={"title": "Number of Customer", "overlaying": "y", "side": "right"},
    title='Quantity and Number of Customer Forecast'
)


fig.show()